## Important: Need to Execute this First Cell Before Continuing

The code in this cell will download a file with a Python scripts from the Internet. Make sure that you have a network connection before executing it.



In [2]:
import requests
with open("hill_cipher.py", 'w') as foo:
    foo.write(requests.get("https://git.io/fj5jZ", timeout = 2).text)
with open("hill_cipher_samples.py", 'w') as foo:
    foo.write(requests.get("https://git.io/fj5jc", timeout = 2).text)
from hill_cipher import *

### Hill cipher
This goal of this notebook is to show how messages can be encrypted and decrypted with the Hill cipher. For simplicity we will assume here that messages that we want to encrypt consist of capital letters A-Z and the space character only.

## Encryption
As an example we will encrypt the message "TOP SECRET"

#### Step 1.  
The first step is to select an invertible matrix that will serve as the encryption key. Here we will use the following matrix $K$
:

In [4]:
K = Matrix([[1, 1, 1], [2, 1, 4], [1, 0, 2]])
K

⎡1  1  1⎤
⎢       ⎥
⎢2  1  4⎥
⎢       ⎥
⎣1  0  2⎦

Since $K$ is
 a $ 3 \times 3$
 matrix, the number of characters in a message we encrypt with it must be divisible by 3. "TOP SECRET" consists of 10 characters (9 letters and a space), so we will add two characters "X" at its end to satisfy this requirement. Thus he whole text we will encrypt will be "TOP SECRETXX":

In [5]:
message = 'TOP SECRETXX'

#### Step 2
 Next, we replace letters in the message with numbers using the following scheme (the underscore stands for the space character):



In [7]:
show_encoding()


  _  A  B  C  D  E  F  G  H  I  J  K  L  M  N  O  P  Q  R  S  T  U  V  W  X  Y  Z
  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26

The function char2num() can be used to perform the letter to number replacement automatically:



In [8]:
numbers = char2num(message)
numbers

[20, 15, 16, 0, 19, 5, 3, 18, 5, 20, 24, 24]

#### Step 3. 
The next step is to split the above list of numbers into vectors with 3 entries each. This can be done as follows:

In [12]:
numbersMatrix = Matrix(numbers).reshape(4,3).T
numbersMatrix

⎡20  0   3   20⎤
⎢              ⎥
⎢15  19  18  24⎥
⎢              ⎥
⎣16  5   5   24⎦

In [13]:
v1 = numbersMatrix[:,0]
v2 = numbersMatrix[:,1]
v3 = numbersMatrix[:,2]
v4 = numbersMatrix[:,3]

In [15]:
v1, v2, v3, v4

⎛⎡20⎤  ⎡0 ⎤  ⎡3 ⎤  ⎡20⎤⎞
⎜⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎟
⎜⎢15⎥, ⎢19⎥, ⎢18⎥, ⎢24⎥⎟
⎜⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎟
⎝⎣16⎦  ⎣5 ⎦  ⎣5 ⎦  ⎣24⎦⎠

We can also do this by just slicing the list.

In [17]:
v1 = Matrix(numbers[0:3])
v2 = Matrix(numbers[3:6])
v3 = Matrix(numbers[6:9])
v4 = Matrix(numbers[9:12])
v1, v2, v3, v4

⎛⎡20⎤  ⎡0 ⎤  ⎡3 ⎤  ⎡20⎤⎞
⎜⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎟
⎜⎢15⎥, ⎢19⎥, ⎢18⎥, ⎢24⎥⎟
⎜⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎟
⎝⎣16⎦  ⎣5 ⎦  ⎣5 ⎦  ⎣24⎦⎠

#### Step 4.
Next, we multiply each vector by the encryption matrix $K$

In [19]:
w1 = K*v1
w2 = K*v2 
w3 = K*v3
w4 = K*v4
w1, w2, w3, w4

⎛⎡51 ⎤  ⎡24⎤  ⎡26⎤  ⎡68 ⎤⎞
⎜⎢   ⎥  ⎢  ⎥  ⎢  ⎥  ⎢   ⎥⎟
⎜⎢119⎥, ⎢39⎥, ⎢44⎥, ⎢160⎥⎟
⎜⎢   ⎥  ⎢  ⎥  ⎢  ⎥  ⎢   ⎥⎟
⎝⎣52 ⎦  ⎣10⎦  ⎣13⎦  ⎣68 ⎦⎠

Note: We could have just applied the encoding matrix to our entire numbersMatrix we defined.

In [21]:
W = K*numbersMatrix
W

⎡51   24  26  68 ⎤
⎢                ⎥
⎢119  39  44  160⎥
⎢                ⎥
⎣52   10  13  68 ⎦

Entries of these new vectors written as a list form the encrypted message.

In [22]:
cipher = list(Matrix([w1, w2, w3, w4]))
cipher


[51, 119, 52, 24, 39, 10, 26, 44, 13, 68, 160, 68]

In [24]:
cipher = list(W.T.reshape(1,12))
cipher

[51, 119, 52, 24, 39, 10, 26, 44, 13, 68, 160, 68]

## Now we need to Decrypt the Message:

Our encrypted message gives us this cipher. Now imagine we have recieved this and do not know what message it holds. How do we uncover it? 

In [25]:
cipher

[51, 119, 52, 24, 39, 10, 26, 44, 13, 68, 160, 68]

### Step 1.

We compute the inverse matrix of $K$. $D = K^{-1}$ is the decrpytion key.

In [27]:
D = K.inv()
D

⎡2   -2  3 ⎤
⎢          ⎥
⎢0   1   -2⎥
⎢          ⎥
⎣-1  1   -1⎦

### Step 2
We split the encrypted message into chunks of 3 characters

In [28]:
u1 = Matrix(cipher[:3])  # a vector consisting of the first 3 numbers of the encrypted message
u2 = Matrix(cipher[3:6]) # a vector consisting of the next 3 numbers of the encrypted message
u3 = Matrix(cipher[6:9]) # and so on...
u4 = Matrix(cipher[9:12])
u1, u2, u3, u4

⎛⎡51 ⎤  ⎡24⎤  ⎡26⎤  ⎡68 ⎤⎞
⎜⎢   ⎥  ⎢  ⎥  ⎢  ⎥  ⎢   ⎥⎟
⎜⎢119⎥, ⎢39⎥, ⎢44⎥, ⎢160⎥⎟
⎜⎢   ⎥  ⎢  ⎥  ⎢  ⎥  ⎢   ⎥⎟
⎝⎣52 ⎦  ⎣10⎦  ⎣13⎦  ⎣68 ⎦⎠

### Step 3
We multiply each vector by the decryption matrix $D$.

In [32]:
z1 = D*u1
z2 = D*u2
z3 = D*u3
z4 = D*u4
z1, z2, z3, z4

⎛⎡20⎤  ⎡0 ⎤  ⎡3 ⎤  ⎡20⎤⎞
⎜⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎟
⎜⎢15⎥, ⎢19⎥, ⎢18⎥, ⎢24⎥⎟
⎜⎢  ⎥  ⎢  ⎥  ⎢  ⎥  ⎢  ⎥⎟
⎝⎣16⎦  ⎣5 ⎦  ⎣5 ⎦  ⎣24⎦⎠

### Step 4 
Combine these into a list of numbers.

In [33]:
numbers = list(Matrix([z1, z2, z3, z4]))
numbers

[20, 15, 16, 0, 19, 5, 3, 18, 5, 20, 24, 24]

### Step 5
Convert the numbers back into letters. 

In [34]:
num2char(numbers)


  20  15  16   0  19   5   3  18   5  20  24  24
   T   O   P   _   S   E   C   R   E   T   X   X



In [35]:
num2char_text_only(numbers)


'TOP_SECRETXX'